In [28]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10002.000000,1.000200e+04,10002.000000,10001.000000,10002.000000,10002.000000,10002.000000,10001.000000,10001.000000,10002.000000,10002.000000
mean,5001.499600,1.569093e+07,650.555089,38.922311,5.012498,76491.112875,1.530194,0.705529,0.514949,100083.331145,0.203759
std,2887.472338,7.193177e+04,96.661615,10.487200,2.891973,62393.474144,0.581639,0.455827,0.499801,57508.117802,0.402812
min,1.000000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,2501.250000,1.562852e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50983.750000,0.000000
50%,5001.500000,1.569073e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.000000,1.000000,100185.240000,0.000000
75%,7501.750000,1.575323e+07,718.000000,44.000000,7.000000,127647.840000,2.000000,1.000000,1.000000,149383.652500,0.000000
max,10000.000000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10002 non-null  int64  
 1   CustomerId       10002 non-null  int64  
 2   Surname          10002 non-null  object 
 3   CreditScore      10002 non-null  int64  
 4   Geography        10001 non-null  object 
 5   Gender           10002 non-null  object 
 6   Age              10001 non-null  float64
 7   Tenure           10002 non-null  int64  
 8   Balance          10002 non-null  float64
 9   NumOfProducts    10002 non-null  int64  
 10  HasCrCard        10001 non-null  float64
 11  IsActiveMember   10001 non-null  float64
 12  EstimatedSalary  10002 non-null  float64
 13  Exited           10002 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 1.1+ MB


In [5]:
#Finding missing values 
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
#Removing null values
data = data.dropna(axis=0)

In [7]:
#Removing column surname
data.drop(columns="Surname", inplace=True)

In [8]:
#Converting category columns into numeric type
cat_cols = data.select_dtypes("object").columns

for col in cat_cols:
    data[col] = pd.factorize(data[col])[0]

In [25]:
data.head()

,RowNumber,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,619,0,0,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,608,1,0,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,502,0,0,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,699,0,0,39.0,1,0.00,2,0.0,0.0,93826.63,0
5,6,15574012,645,1,1,44.0,8,113755.78,2,1.0,0.0,149756.71,1


In [10]:
X = data.drop(columns="Exited")
y = data["Exited"]

In [11]:
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.3)

##### Model

In [12]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [13]:
y_predictions = model.predict(x_test)

##### Metrics

In [20]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [21]:
print(f"Classification Report: \n {classification_report(y_test,y_predictions)}")

Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.96      0.87      2374
           1       0.28      0.05      0.09       626

    accuracy                           0.77      3000
   macro avg       0.54      0.51      0.48      3000
weighted avg       0.69      0.77      0.71      3000



In [22]:
print(f"Confusion Matrix:\n {confusion_matrix(y_test,y_predictions)}")
print(f"Accuracy: {accuracy_score(y_test,y_predictions)}")

Confusion Matrix:
 [[2286   88]
 [ 592   34]]
Accuracy: 0.7733333333333333


#### Grid Search Cross Validation

In [25]:
from sklearn.model_selection import GridSearchCV

In [24]:
hyperparameters = {'penalty':['l1', 'l2', 'elasticnet'],
                  'C':[0.01,0.1,1.0],
                   'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

In [26]:
model_GSCV = GridSearchCV(estimator=LogisticRegression(), param_grid=hyperparameters, cv=3)

In [29]:
model_GSCV.fit(x_train,y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1.0],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']})

In [30]:
model_GSCV.best_params_

{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}

In [31]:
model_LR = LogisticRegression(C=0.1,penalty='l1',solver='liblinear')

In [32]:
model_LR.fit(x_train,y_train)

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')

In [34]:
predictions = model_LR.predict(x_test)

In [35]:
print(f"Classification Report: \n {classification_report(y_test,predictions)}")

Classification Report: 
               precision    recall  f1-score   support

           0       0.82      0.96      0.89      2374
           1       0.61      0.22      0.33       626

    accuracy                           0.81      3000
   macro avg       0.72      0.59      0.61      3000
weighted avg       0.78      0.81      0.77      3000



In [36]:
print(f"Confusion Matrix:\n {confusion_matrix(y_test,predictions)}")
print(f"Accuracy: {accuracy_score(y_test,predictions)}")

Confusion Matrix:
 [[2284   90]
 [ 486  140]]
Accuracy: 0.808
